# Enviroment Setting

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/AN2DL-challenge-2022-polimi/

/gdrive/.shortcut-targets-by-id/1Pu-dfIBpuqbYCNKRZsNFDRCnCWgmOpf1/AN2DL-challenge-2022-polimi


In [ ]:
import os
import random
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

import tensorflow as tf
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
tf.get_logger().setLevel('ERROR')

# Splitting

In [ ]:
!pip install split-folders
import splitfolders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!find ./Dataset/training_data_final/ -type d -print -exec sh -c "ls {} | wc -l | head -n1" \;

./Dataset/training_data_final/
8
./Dataset/training_data_final/Species7
537
./Dataset/training_data_final/Species8
508
./Dataset/training_data_final/Species1
186
./Dataset/training_data_final/Species6
222
./Dataset/training_data_final/Species3
515
./Dataset/training_data_final/Species4
511
./Dataset/training_data_final/Species5
531
./Dataset/training_data_final/Species2
532


In [ ]:
splitfolders.ratio("Dataset/training_data_final", output="Dataset/splitted_data_new",  #split the dataset without balancing the class
                   seed=seed, ratio=(.8, .2), group_prefix=None, move=False)

In [ ]:
splitfolders.fixed("Dataset/training_data_final", output="Dataset/splitted_data_balanced_new", #split the dataset with class balancing
    seed=seed, fixed=(50), oversample=True)

In [ ]:
splitfolders.fixed("Dataset/training_data_final", output="Dataset/splitted_data_balanced_val", #split the dataset with class balancing
    seed=seed, fixed=(50), oversample=False)

Copying files: 3542 files [01:24, 41.97 files/s]


In [ ]:
!find ./Dataset/splitted_data_balanced_val/ -type d -print -exec sh -c "ls {} | wc -l | head -n1" \; #verfify if it's all right

./Dataset/splitted_data_balanced_val/
2
./Dataset/splitted_data_balanced_val/train
8
./Dataset/splitted_data_balanced_val/train/Species7
487
./Dataset/splitted_data_balanced_val/train/Species8
458
./Dataset/splitted_data_balanced_val/train/Species1
136
./Dataset/splitted_data_balanced_val/train/Species6
172
./Dataset/splitted_data_balanced_val/train/Species3
465
./Dataset/splitted_data_balanced_val/train/Species4
461
./Dataset/splitted_data_balanced_val/train/Species5
481
./Dataset/splitted_data_balanced_val/train/Species2
482
./Dataset/splitted_data_balanced_val/val
8
./Dataset/splitted_data_balanced_val/val/Species7
50
./Dataset/splitted_data_balanced_val/val/Species8
50
./Dataset/splitted_data_balanced_val/val/Species1
50
./Dataset/splitted_data_balanced_val/val/Species6
50
./Dataset/splitted_data_balanced_val/val/Species3
50
./Dataset/splitted_data_balanced_val/val/Species4
50
./Dataset/splitted_data_balanced_val/val/Species5
50
./Dataset/splitted_data_balanced_val/val/Species2
50


# Keras cv augmentation

In [ ]:
!pip install keras_cv
import keras_cv
!pip install imgaug
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
input_shape = (96, 96, 3)
target_size=(96,96)
classes = 8
epochs = 100
BATCH_SIZE = 16
AUTO = tf.data.AUTOTUNE
IMG_SIZE = 96

In [ ]:
# Setting dataset directories
splitted_data_dir = os.path.join('Dataset/splitted_data_balanced_new')
training_dir = os.path.join(splitted_data_dir + '/train')
validation_dir = os.path.join(splitted_data_dir + '/val')

In [ ]:
# Labels of the dataset for classification
labels = ['Species1',
          'Species2',
          'Species3',
          'Species4',
          'Species5',
          'Species6',
          'Species7',
          'Species8']

In [ ]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.

from tensorflow.keras.applications.convnext import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=target_size,
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=seed)
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=target_size,
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE,
                                               shuffle=False,
                                               seed=seed)

In [ ]:
x_train=np.concatenate([train_gen.next()[0] for i in range(train_gen.__len__())])
y_train=np.concatenate([train_gen.next()[1] for i in range(train_gen.__len__())])
print(x_train.shape)
print(y_train.shape)

x_val=np.concatenate([valid_gen.next()[0] for i in range(valid_gen.__len__())])
y_val=np.concatenate([valid_gen.next()[1] for i in range(valid_gen.__len__())])
print(x_val.shape)
print(y_val.shape)

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.image.convert_image_dtype(image, tf.float32) / 255.0
    return image, label

In [ ]:
train_ds_one = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
)
train_ds_two = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
)

train_ds_simple = tf.data.Dataset.from_tensor_slices((x_train, y_train))

train_ds_simple = (
    train_ds_simple.map(preprocess_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Combine two shuffled datasets from the same training data.
train_ds = tf.data.Dataset.zip((train_ds_one, train_ds_two))


val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))

val_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .map(preprocess_image, num_parallel_calls=AUTO)
)

In [ ]:
def sample_beta_distribution_cutmix(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


@tf.function
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = IMG_SIZE * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = IMG_SIZE * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=IMG_SIZE, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, IMG_SIZE)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, IMG_SIZE)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, IMG_SIZE)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, IMG_SIZE)

    target_h = bby2 - boundaryy1
    if target_h == 0:
        target_h += 1

    target_w = bbx2 - boundaryx1
    if target_w == 0:
        target_w += 1

    return boundaryx1, boundaryy1, target_h, target_w


@tf.function
def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two

    alpha = [0.25]
    beta = [0.25]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution_cutmix(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, IMG_SIZE, IMG_SIZE
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (IMG_SIZE * IMG_SIZE)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    label = lambda_value * label1 + (1 - lambda_value) * label2
    return image, label

In [ ]:
def sample_beta_distribution_mixup(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

@tf.function
def mix_up(train_ds_one, train_ds_two, alpha=0.2):
    # Unpack two datasets
    images_one, labels_one = train_ds_one
    images_two, labels_two = train_ds_two
    batch_size = tf.shape(images_one)[0]

    # Sample lambda and reshape it to do the mixup
    l = sample_beta_distribution_mixup(batch_size, alpha, alpha)
    x_l = tf.reshape(l, (batch_size, 1, 1, 1))
    y_l = tf.reshape(l, (batch_size, 1))

    # Perform mixup on both images and labels by combining a pair of images/labels
    # (one from each dataset) into one image/label
    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return (images, labels)

In [ ]:
rand_aug = iaa.RandAugment(n=3, m=7)


def augment(images):
    # Input to `augment()` is a TensorFlow tensor which
    # is not supported by `imgaug`. This is why we first
    # convert it to its `numpy` variant.
    images = tf.cast(images, tf.uint8)
    return rand_aug(images=images.numpy())

In [ ]:
#Rand augmentation training set
train_ds_randaug = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(1024)
    .batch(BATCH_SIZE)
    .map(lambda x, y: (tf.py_function(augment, [x], [tf.float32])[0], y), num_parallel_calls=AUTO)
    .prefetch(AUTO)
)

In [ ]:
sample_images, _ = next(iter(train_ds_randaug))
plt.figure(figsize=(10, 10))
for i, image in enumerate(sample_images[:9]):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().astype("int"))
    plt.axis("off")

In [ ]:
#Cut mix training set
train_ds_cmu = (
    train_ds.shuffle(1024)
    .map(cutmix, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [ ]:
# Visualization after cutmix
image_batch, label_batch = next(iter(train_ds_cmu))
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.title(labels[np.argmax(label_batch[i])])
    plt.imshow(image_batch[i])
    plt.axis("off")

In [ ]:
#Mix up training set
train_ds_mu = train_ds.map(
    lambda ds_one, ds_two: mix_up(ds_one, ds_two, alpha=0.2), num_parallel_calls=AUTO
)

In [ ]:
#Visualization after mixup
sample_images, sample_labels = next(iter(train_ds_mu))
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(zip(sample_images[:9], sample_labels[:9])):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().squeeze())
    print(label.numpy().tolist())
    plt.axis("off")

In [ ]:
train_ds_all_aug = tf.data.Dataset.zip((train_ds_randaug, train_ds_cmu, train_ds_mu))
combined_dataset1 = train_ds_cmu.concatenate(train_ds_mu)
combined_dataset = combined_dataset1.concatenate(train_ds_randaug)

# Ensembling

In [ ]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, w3, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]

In [ ]:
Standarnd_CNN_model = tfk.models.load_model('Standard_NoAug_class_weight_Best')
Resnet101_model = tfk.models.load_model('resnet101')
ConvnextLarge_model = tfk.models.load_model('convnextlarge') 

In [ ]:
Standarnd_CNN_model._name = 'Standarnd_CNN_model'
Standarnd_CNN_model.summary()
Resnet101_model._name = 'Resnet101_model'
Resnet101_model.summary()
ConvnextLarge_model._name = 'ConvnextLarge_model'
ConvnextLarge_model.summary()

In [ ]:
models = [Standarnd_CNN_model, Resnet101_model, ConvnextLarge_model]
model_input = tf.keras.Input(shape=(96, 96, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = WeightedAverageLayer(0.25, 0.25, 0.5)(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
ensemble_model.save('Ensemble_model')